# Verifying Kinetics Models: Part 1 - Software Infrastructure

Testing provides a way to find defects in products such as cars televisions, food, and software.

There are two broad objectives for testing. **Validation** determines if the product provides a useful purpose. In terms of kinetics models this means that the model is accurate, and is useful for its intended objectives. For example, to a model for predicting ICU usage for COVID-19 patients might involve: (a) checking its predictions vs. observed future data and (b) estimating the mortality, morbidity, and cost implications resulting from inaccuracies of the model. In general, validation is context and discipline specific.

**Verification** is about determining if the product performs according to its specification. For kinetics models, this means that the model dynamics of the model are consistent with what is intended (even if these are not the *correct* dynamics). Validation software engineering typically takes the form of unittests, codes that detect errors in the functioning of software components.

This tutorial focuses on verification of kinetics models, ensuring that the intended dynamics are produced by the model. This tutorial is divided into two parts. The first part describes the software setup for doing verification of kinetics models in Jupyter Notebooks. The second part describes an approach to writing kinetics tests using this software setup.

In [1]:
import numpy as np
import tellurium as te
from teUtils.named_timeseries import NamedTimeseries, TIME
from teUtils.timeseries_plotter import TimeseriesPlotter, PlotOptions

## Lecture

### Motivation and Background

Testing is the process by which you exercise your code to determine if it performs as expected. The code you are testing is referred to as the code under test.

There are two parts to writing tests.
1. invoking the code under test so that it is exercised in a particular way;
1. evaluating the results of executing code under test to determine if it behaved as expected.

The collection of tests performed are referred to as the test cases.

### Testing in a Jupyter Notebook

In [2]:
 simple_model = '''
    model example1
      S1 -> S2; k1*S1
      S1 = 10
      S2 = 0
      k1 = 0.1
    end
    '''

In [3]:
# Simulation runner
def runSimulation(model):
    """
    Runs a simulation for an antimony model
    
    Parameters
    ----------
    model: str
    
    Returns
    -------
    NamedTimeseries
    """
    rr = te.loada(model)
    data = rr.simulate()
    return NamedTimeseries(named_array=data)

In [4]:
# Tests return True if passed and False if fail.
def test1(ts):
    return len(ts) > 0

def test2(ts):
    s1 = ts["S1"]
    return s1[0] == 10

In [5]:
# Test runner
def runTests(model):
    ts = runSimulation(model)
    is_ok = True
    for test in [test1, test2]:
        is_ok = is_ok and test(ts)
    if is_ok:
        print("OK.")
    else:
        print("Problems encountered in model.")

In [6]:
runTests(simple_model)

OK.


## Breakout

In the following, you will create tests for a new model called ``model``.

In [7]:
new_model = '''
        # Reactions   
        J1: S1 -> S2; k1*S1
        J2: S2 -> S3; k2*S2
        J3: S3 -> S4; k3*S3
        J4: S4 -> S5; k4*S4
        J5: S5 -> S6; k5*S5;
        # Species initializations     
        k1 = 1; k2 = 2; k3 = 3; k4 = 4; k5 = 5;
        S1 = 10; S2 = 0; S3 = 0; S4 = 0; S5 = 0; S6 = 0;
        '''

#### Run the existing tests for this model. Did the model pass?

In [8]:
runTests(new_model)

OK.


#### Improve the tests by including print statements that describe the error.

In [9]:
# Tests
def test1(ts):
    if len(ts) > 0:
        return True
    else:
        print("No data produced by simulation.")
        return False
    
def test2(ts):
    s1 = ts["S1"]
    if s1[0] == 10:
        return True
    else:
        print("Initial value of S1 should be 10!")
        return False

In [10]:
runTests(new_model)

OK.


#### Fix the error in the model and re-run the tests.

In [11]:
# Fix error in model

#### Write additional tests.
1. The maxium value of S1 is at 0.
1. The maximum value of S6 is its last value.

In [12]:
def test3(ts):
    """
    Check that the maximum value of S1 is at time 0.
    """
    s1 = ts["S1"]
    if s1[0] == max(s1):
        return True
    else:
        print("The initial value of S1 should be 0!")
        return False
    
def test4(ts):
    """
    Check that the maximum value of s6 is at the end of the simulation.
    """
    s6 = ts["S6"]
    if s6[-1] == max(s6):
        return True
    else:
        print("The maximum value of S6 should be at the last time!")
        return False

In [13]:
def newRunTests(model):
    ts = runSimulation(model)
    is_ok = True
    for test in [test1, test2, test3, test4]:
        is_ok = is_ok and test(ts)
    if is_ok:
        print("OK.")
    else:
        print("Problems encountered in model.")

In [14]:
newRunTests(new_model)

OK.


#### Advanced
A problem with the above infrastructre is that we need to revise the test runner every time we add or remove a test. Having to make coordinated changes in software is extremely bad practice and is a common cause of software errors.

Another solution is to have a "test container". In python, this is done by creating a python class in which tests reside. We won't provide details about python classes here. Rather, you can just use the code below. The main changes are the ``__init__`` function at the top, the arguments to the test, and the use of ``self.ts`` to access the timeseries. The ``run`` function runs all functions in ``TestContainer`` that begin with ``test``.

In [15]:
class TestContainer(object):
    
    def __init__(self, model):
        ts = runSimulation(model)
        self.ts = ts
    
    def test1(self):
        if len(self.ts) > 0:
            return True
        else:
            print("No data produced by simulation.")
            return False

    def test2(self):
        s1 = self.ts["S1"]
        if s1[0] == 10:
            return True
        else:
            print("Initial value of S1 should be 10!")
            return False
        
    def run(self):
        is_ok = True
        for item in dir(self):
            if item[0:4] == "test":
                # Construct the function call
                func = "self.%s()" % item
                is_ok = is_ok and eval(func)
        if is_ok:
            print("OK.")
        else:
            print("Problems encountered in model.")

To run the tests, you do the follow:

In [16]:
tester = TestContainer(simple_model)
tester.run()

OK.


Now create an error by modifying the initial value of S1 to show that this works.

Last, add ``test3`` and ``test4`` to ``TestContainer``. We don't have to modify any other codes to include these tests.